In [1]:
import torch
from torch import nn

In [45]:

#export
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, ε:float=0.1, reduction='mean'):
        super().__init__()
        self.ε,self.reduction = ε,reduction
    
    def forward(self, output, target):
        c = output.size()[-1]
        log_preds = F.log_softmax(output, dim=-1)
        loss = reduce_loss(-log_preds.sum(dim=-1), self.reduction)
        nll = F.nll_loss(log_preds, target, reduction=self.reduction)
        print(loss/c, nll)
        return lin_comb(loss/c, nll, self.ε)

In [46]:
x = torch.rand((64,16))
y = torch.ones(64).long()

In [47]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(16,10)
        
    def forward(self, x):
        return self.lin(x)

In [48]:
m = Net()

In [49]:
y_pred = m(x)

In [60]:
y_pred[:,1] = 10

In [50]:
cross_entropy = nn.CrossEntropyLoss()

In [51]:
import torch.nn.functional as F

In [52]:
F.cross_entropy(y_pred, y)

tensor(2.2694, grad_fn=<NllLossBackward>)

In [53]:
cross_entropy(y_pred, y)

tensor(2.2694, grad_fn=<NllLossBackward>)

# label

In [54]:
label_smoothing = LabelSmoothingCrossEntropy()

In [62]:
def reduce_loss(loss, reduction='mean'):
    return loss.mean() if reduction=='mean' else loss.sum() if reduction=='sum' else loss

def lin_comb(a, b, t):
    return a*t + (1-t)*b

In [63]:
label_smoothing(y_pred,y)

tensor(8.9866, grad_fn=<DivBackward0>) tensor(0.0004, grad_fn=<NllLossBackward>)


tensor(0.8991, grad_fn=<AddBackward0>)